# McStasRead demo

In [ ]:
%matplotlib widget

In [ ]:
from read_example import make_instrument, plot

In [ ]:
instr = make_instrument()
instr.show_parameters()

In [ ]:
instr.set_parameters(wavelength=1.8, delta_wavelength=1.3)
instr.settings(ncount=1E6)

In [ ]:
instr.show_diagram()

### Run simulation

In [ ]:
data = instr.backengine()

In [ ]:
data

### Histogram data to display

In [ ]:
hist_data = [mon.make_2d(mon.variables[1], "y") for mon in data]

In [ ]:
import mcstasscript as ms
ms.make_sub_plot(hist_data)

In [ ]:
file_path = data[0].original_data_location
print(file_path)

### Show list of components found in NeXus file

In [ ]:
import mcstastox

with mcstastox.Read(file_path) as loaded_data:
    loaded_data.show_components()

#### Show list of these with available data

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    loaded_data.show_components_with_data()

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    variables = loaded_data.get_component_variables("Square_1")
    print(variables)

#### Show list of these with geometry information

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    loaded_data.show_components_with_geometry()

#### Show monitors with pixel ID's

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    loaded_data.show_components_with_ids()

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    print(loaded_data.get_components_with_ids())

### Get local pixel coordinates for a given component

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    coordinates = loaded_data.get_component_local("Square_1")

plot(coordinates)

### Get global coordinates for a given component

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    coordinates = loaded_data.get_component_global("Square_1")

plot(coordinates)

### Get sample position as well

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    coordinates = loaded_data.get_component_global("Square_2")
    sample_pos = loaded_data.get_global_component_coordinates("sample_position")

plot(coordinates, points=[sample_pos])

### Get pixel coordinates for list of monitors

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    coordinates = loaded_data.get_id_to_global_coordinates(component_name=["Square_2", "Banana_1", "Banana_2"])
    sample_pos = loaded_data.get_global_component_coordinates("sample_position")

plot(coordinates, points=[sample_pos])

### Include source and get all pixel coordinates

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    coordinates = loaded_data.get_id_to_global_coordinates()
    sample_pos = loaded_data.get_global_component_coordinates("sample_position")
    source_pos = loaded_data.get_global_component_coordinates("source")

plot(coordinates, points=[sample_pos, source_pos])

### Export to (simple) Scipp object

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    scipp_object = loaded_data.export_scipp_simple(source_name="source",
                                                   sample_name="sample_position",
                                                   component_name="Banana_2")

In [ ]:
scipp_object

### Plot event positions

In [ ]:
import plopp as pp

pp.scatter3d(scipp_object[0::3], pos='position', size=0.02, cbar=True, norm="linear")

### Perform coordinate transforms

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.graph.tof import elastic

# McStas provides absolute time, not time of flight
scipp_object.coords["tof"] = scipp_object.coords["t"]

graph = {**beamline(scatter=True), **elastic("tof")}

In [ ]:
scipp_object = scipp_object.transform_coords("dspacing", graph=graph)

In [ ]:
scipp_object.hist(dspacing=150).plot(norm="log")

## Load all data in simple scipp format

In [ ]:
import mcstastox
with mcstastox.Read(file_path) as loaded_data:
    all_data = loaded_data.export_scipp_simple(source_name="source",
                                                   sample_name="sample_position")

all_data

In [ ]:
pp.scatter3d(all_data[0::3], pos='position', size=0.02, cbar=True, norm="log")

### Convert to d-space and compare
Lets compare data from all the detectors with the previous scipp object that only contained one.

In [ ]:
all_data.coords["tof"] = all_data.coords["t"]
all_data = all_data.transform_coords("dspacing", graph=graph)

pp.plot({"Only banana 2": scipp_object.hist(dspacing=200),
         "All data": all_data.hist(dspacing=200)}, norm="log")

### Also access to data directly
Its possible to get access to the data directly if necessary, this is demonstrated here.

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    raw_event_data = loaded_data.get_event_data(variables=["id", "t"], component_name="Square_2")
    pos, rot = loaded_data.get_component_placement(component_name="Square_2")

print("Raw event data")
print(raw_event_data["id"])
print(raw_event_data["t"])

print("Component placement")
print(pos)
print(rot)

In [ ]:
with mcstastox.Read(file_path) as loaded_data:
    raw_event_data = loaded_data.get_event_data(variables=["id", "t", "L"], component_name="Square_1")

### Add additional data to a scipp object
We can add data read directly to the scipp object, here the actual wavelength recorded by McStas.

In [ ]:
import scipp as sc

with mcstastox.Read(file_path) as loaded_data:
    sq1 = loaded_data.export_scipp_simple(source_name="source", sample_name="sample_position", component_name="Square_1")

sq1.coords["sim_wavelength"] = sc.array(dims=["events"], values=raw_event_data["L"], unit="Å")

sq1.coords["tof"] = sq1.coords["t"]
sq1 = sq1.transform_coords("wavelength", graph=graph)
sq1.coords["wavelength_ratio"] = (sq1.coords["sim_wavelength"] - sq1.coords["wavelength"])/sq1.coords["sim_wavelength"]

In [ ]:
sq1

### Plotting comparison between actual and calculated wavelength

In [ ]:
w_start = 0.5
w_end=3.2
w_steps=401
wavs = sc.linspace("wavelength", w_start, w_end, w_steps, unit="angstrom")
sim_wavs = sc.linspace("sim_wavelength", w_start, w_end, w_steps, unit="angstrom")

pp.plot({
    "calculated wavelength": sq1.hist(wavelength=wavs),
    "true wavelength": sq1.hist(sim_wavelength=sim_wavs).rename(sim_wavelength="wavelength")
}) / sq1.hist(wavelength_ratio=200).plot()